# LSTM AutoEncoder 실습

https://machinelearningmastery.com/lstm-autoencoders/


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, models, layers, optimizers, utils

## Reconstruction LSTM AutoEncoder

In [3]:
# define input sequence
seq = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq)
seq = seq.reshape((1, n_in, 1))

# define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in, 1)))
model.add(layers.RepeatVector(n_in))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq, seq, epochs=300, verbose=0)

# predict
yhat = model.predict(seq)
yhat

array([[[0.10480094],
        [0.20075469],
        [0.2992439 ],
        [0.39900646],
        [0.49926344],
        [0.59960264],
        [0.6998814 ],
        [0.8001507 ],
        [0.9006021 ]]], dtype=float32)

## Prediction LSTM AutoEncoder

In [6]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define model
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in, 1)))
model.add(layers.RepeatVector(n_out))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)

# predict
yhat = model.predict(seq_in)
yhat

array([[[0.16838059],
        [0.29353157],
        [0.40570223],
        [0.50761384],
        [0.60400295],
        [0.69897115],
        [0.79663056],
        [0.9015523 ]]], dtype=float32)

In [14]:
# define input sequence
seq_in = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# reshape input into [samples, timesteps, features]
n_in = len(seq_in)
seq_in = seq_in.reshape((1, n_in, 1))

# prepare output sequence
seq_out = seq_in[:, 1:, :]
n_out = n_in - 1

# define encoder
visible = layers.Input(shape=(n_in, 1))
encoder = layers.LSTM(100, activation='relu')(visible)
# define reconstruct decoder
decoder1 = layers.RepeatVector(n_in)(encoder)
decoder1 = layers.LSTM(100, activation='relu', return_sequences=True)(decoder1)
decoder1 = layers.TimeDistributed(layers.Dense(1))(decoder1)
# define predict decoder
decoder2 = layers.RepeatVector(n_out)(encoder)
decoder2 = layers.LSTM(100, activation='relu', return_sequences=True)(decoder2)
decoder2 = layers.TimeDistributed(layers.Dense(1))(decoder2)
# concat model
model = Model(inputs=visible, outputs=[decoder1, decoder2])
model.compile(optimizer='adam', loss='mse')
# utils.plot_model(model, show_shapes=True, to_file='composite_lstm_autoencoder.png')

# fit model
model.fit(seq_in, [seq_in, seq_out], epochs=300, verbose=0)

# predict
y_hat = model.predict(seq_in)
y_hat

[array([[[0.1115096 ],
         [0.20743433],
         [0.30335915],
         [0.3996057 ],
         [0.4966244 ],
         [0.59498614],
         [0.69538325],
         [0.7986429 ],
         [0.9057488 ]]], dtype=float32), array([[[0.16606757],
         [0.28853485],
         [0.40248707],
         [0.5094125 ],
         [0.6108996 ],
         [0.7076266 ],
         [0.8002331 ],
         [0.88931644]]], dtype=float32)]